In [31]:
import pandas as pd
import json
import random
import time
import hashlib
import os
import uuid

In [32]:
CITIES = [
    'Ciudad de México',
    'Guadalajara',
    'Queretaro',
    'Monterrey',
    'Puebla'
]

PAYMENT_ONLINE = ['Debit', 'Credit']

OS_DEVICE = ['WEB','ANDROID', 'IOS']

INITIAL_EVENT_FLOW = 'LAUNCH_APP'
SECOND_EVENT_FLOW = ['HOME','EXIT_APP','HOME','HOME']
THIRD_EVENT_FLOW = ['GO_TO_CATEGORY','GO_TO_CATEGORY','GO_TO_CATEGORY','EXIT_APP']
EVENT_CATEGORY_FLOW= ['LIQUOR','PHARMACY','TECHNOLOGY','ELECTRO_DOMESTIC','BABY','CLOTHES']
FINAL_EVENT_FLOW = ['PURCHASE','PURCHASE','PURCHASE','EXIT_APP','EXIT_APP','PURCHASE']


STORE_COORDS_BY_CITY = {
    'Ciudad de México':(19.372879, -99.049378),
    'Guadalajara':(20.690072, -103.301842),
    'Queretaro':(20.606305, -100.412364),
    'Monterrey':(25.713272, -100.277447),
    'Puebla':(18.973534, -98.252895),
    
}

#### Simulate user app interaction

In [33]:
def simulate_event_user_app():
    simulation = {
        'payment' : random.choice(PAYMENT_ONLINE),
        'os_device' : random.choice(OS_DEVICE),
        'city' : random.choice(CITIES),
        'status' : 'UNCONVERTED',
        'order_type' : 'USER_VISIT',
        'event_1': INITIAL_EVENT_FLOW,
        'event_2': random.choice(SECOND_EVENT_FLOW),
        'event_3': random.choice(THIRD_EVENT_FLOW),
        'last_event': None

    }

    if simulation['event_2'] == 'HOME':
        if simulation['event_3'] == 'GO_TO_CATEGORY':
            simulation['last_event'] = random.choice(EVENT_CATEGORY_FLOW)
            final_event = random.choice(FINAL_EVENT_FLOW)
            if final_event == 'PURCHASE':
                simulation['status'] = 'COMPLETED'
                simulation['order_type'] = 'PURCHASE'
            else:
                simulation['payment'] = None
                simulation['last_event'] = 'HOME'
        else:
           simulation['payment'] = None
           simulation['last_event'] = 'HOME'
    else:
        simulation['payment'] = None
        simulation['last_event'] = 'LAUNCH_APP'
        simulation['event_3'] = None

    return simulation

def get_store_location(city:str):
    return STORE_COORDS_BY_CITY[city]


def create_users_bank(num_users):
    users = []
    for i in range(num_users):
        user = {
            'created_at' : pd.to_datetime('today').strftime('%Y-%m-%d %H:%M:%S'),
            'user_id' :  str(uuid.uuid4())
        }
        users.append(user)
    return users

In [34]:
user_random_interaction = simulate_event_user_app()
user_random_interaction

{'payment': 'Credit',
 'os_device': 'WEB',
 'city': 'Guadalajara',
 'status': 'COMPLETED',
 'order_type': 'PURCHASE',
 'event_1': 'LAUNCH_APP',
 'event_2': 'HOME',
 'event_3': 'GO_TO_CATEGORY',
 'last_event': 'PHARMACY'}

In [35]:
users = create_users_bank(1000)
users[:10]

[{'created_at': '2024-01-17 16:10:47',
  'user_id': '45635ca9-7026-401a-91ef-e59eef0f1c97'},
 {'created_at': '2024-01-17 16:10:47',
  'user_id': 'e969f136-c1a8-41c3-b335-2da7dba3e6b8'},
 {'created_at': '2024-01-17 16:10:47',
  'user_id': '1ca5b4a0-3933-4cf4-bcac-8fde1b1c22b6'},
 {'created_at': '2024-01-17 16:10:47',
  'user_id': '70df12be-441b-48bd-8ea7-344aed7b0e59'},
 {'created_at': '2024-01-17 16:10:47',
  'user_id': '93ca45ab-bf2f-48a2-af66-f0530724bcf6'},
 {'created_at': '2024-01-17 16:10:47',
  'user_id': 'e3e3ef2e-157e-49b0-9a62-cd075f093f32'},
 {'created_at': '2024-01-17 16:10:47',
  'user_id': '92ad92de-abcf-49cf-a8c3-0316a07279e3'},
 {'created_at': '2024-01-17 16:10:47',
  'user_id': 'beb52a57-7a56-4b1f-9d91-682bceb06b26'},
 {'created_at': '2024-01-17 16:10:47',
  'user_id': 'b8cedb53-1ff5-447e-aded-fc911aed21c3'},
 {'created_at': '2024-01-17 16:10:47',
  'user_id': '85c38223-0911-4388-b582-a5a5153f42d7'}]

In [36]:
USERS = create_users_bank(1000)

In [37]:
def get_users_simulation_app(num_simulations:int):
    x = 0
    purchases_simulation = []
    while x < num_simulations:
        date = pd.to_datetime('today').strftime('%Y-%m-%d %H:%M:%S')
        user_simulation = simulate_event_user_app()
        user_id = random.choice(USERS)['user_id']
        purchase = {
            'user_id':user_id,
            'created_at':date,
            'payment_method' : user_simulation['payment'],
            'os_device' : user_simulation['os_device'],
            'city' : user_simulation['city'],
            'latitude': STORE_COORDS_BY_CITY[user_simulation['city']][0],
            'longitude': STORE_COORDS_BY_CITY[user_simulation['city']][1],
            'status' : user_simulation['status'],
            'order_type' : user_simulation['order_type'],
            'event_1': user_simulation['event_1'],
            'event_2': user_simulation['event_2'],
            'event_3': user_simulation['event_3'],
            'last_event': user_simulation['last_event'],
        }
        purchases_simulation.append(pd.DataFrame(purchase,index=[x]))
        x += 1
        time.sleep(random.choice([1,1,5,2]))
    return purchases_simulation

In [38]:
purchases_simulation = get_users_simulation_app(10)
df_purchases_simulation = pd.concat(purchases_simulation, ignore_index=True)
df_purchases_simulation

,user_id,created_at,payment_method,os_device,city,latitude,longitude,status,order_type,event_1,event_2,event_3,last_event
0,34216542-2e22-4dc6-a078-f6c8856bd4f4,2024-01-17 16:10:47,Debit,IOS,Queretaro,20.606305,-100.412364,COMPLETED,PURCHASE,LAUNCH_APP,HOME,GO_TO_CATEGORY,LIQUOR
1,e49e4eff-b092-465c-919d-f304399b2826,2024-01-17 16:10:48,None,IOS,Puebla,18.973534,-98.252895,UNCONVERTED,USER_VISIT,LAUNCH_APP,EXIT_APP,None,LAUNCH_APP
2,dced38c3-2a80-450b-aaec-b9e62caabf15,2024-01-17 16:10:49,Credit,IOS,Monterrey,25.713272,-100.277447,COMPLETED,PURCHASE,LAUNCH_APP,HOME,GO_TO_CATEGORY,ELECTRO_DOMESTIC
3,b928cb62-9be6-49e3-a03d-50b97e005353,2024-01-17 16:10:54,Credit,IOS,Queretaro,20.606305,-100.412364,COMPLETED,PURCHASE,LAUNCH_APP,HOME,GO_TO_CATEGORY,CLOTHES
4,93b15981-53dd-4abb-8953-9c716c97b956,2024-01-17 16:10:55,None,IOS,Monterrey,25.713272,-100.277447,UNCONVERTED,USER_VISIT,LAUNCH_APP,EXIT_APP,None,LAUNCH_APP
5,fd677c0d-d56c-4865-8b20-a0b127092547,2024-01-17 16:10:56,None,IOS,Guadalajara,20.690072,-103.301842,UNCONVERTED,USER_VISIT,LAUNCH_APP,EXIT_APP,None,LAUNCH_APP
6,09d7a734-20ec-4ddf-84b9-e4afa9f8c373,2024-01-17 16:11:01,None,WEB,Ciudad de México,19.372879,-99.049378,UNCONVERTED,USER_VISIT,LAUNCH_APP,HOME,EXIT_APP,HOME
7,39de6411-0feb-431e-9968-0f5478076ca4,2024-01-17 16:11:03,Debit,WEB,Ciudad de México,19.372879,-99.049378,COMPLETED,PURCHASE,LAUNCH_APP,HOME,GO_TO_CATEGORY,CLOTHES
8,39de6411-0feb-431e-9968-0f5478076ca4,2024-01-17 16:11:04,None,WEB,Guadalajara,20.690072,-103.301842,UNCONVERTED,USER_VISIT,LAUNCH_APP,HOME,GO_TO_CATEGORY,HOME
9,60010386-8f49-47bb-a59f-f0c545476782,2024-01-17 16:11:06,None,ANDROID,Puebla,18.973534,-98.252895,UNCONVERTED,USER_VISIT,LAUNCH_APP,HOME,GO_TO_CATEGORY,HOME


In [39]:
df_purchases_simulation['payment_method'].value_counts()

payment_method
Debit     2
Credit    2
Name: count, dtype: int64

In [41]:
df_purchases_simulation['status'].value_counts()

status
UNCONVERTED    6
COMPLETED      4
Name: count, dtype: int64

In [43]:
df_purchases_simulation['os_device'].value_counts()

os_device
IOS        6
WEB        3
ANDROID    1
Name: count, dtype: int64

In [44]:
df_purchases_simulation['city'].value_counts()

city
Queretaro           2
Puebla              2
Monterrey           2
Guadalajara         2
Ciudad de México    2
Name: count, dtype: int64